# Úkol č. 2 - předzpracování dat a binární klasifikace (do 7. prosince)

  * V rámci tohoto úkolu se musíte vypořádat s příznaky, které jsou různých typů.
  * Před tím, než na nich postavíte predikční model, je třeba je nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na trénovací, testovací a případně i validační množinu.
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí testovací množiny. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor(y) s predikcemi pro vyhodnocovací data.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

In [1]:
import math
import pandas as pd
import numpy as np
np.set_printoptions(precision=1, suppress=True)
from sklearn.neighbors import KNeighborsRegressor

In [2]:
data = pd.read_csv('data.csv')
#display(data.shape)
#display(data.head())
#display(data.nunique())

zahodim priznaky:
    name protoze je skoro unikatni pro kazdeho cloveka;
    ticket protoze cislo listku nema zadny vliv na preziti;
    home.dest ze stejneho duvodu

In [3]:
df = data.drop(columns = ['ID', 'name', 'ticket', 'home.dest'])
string_cols = df.select_dtypes(['object']).columns

for col in string_cols:
    df[col] = df[col].astype('category')
df[string_cols] = df[string_cols].apply(lambda x: x.cat.codes)

xdata = df.drop(columns = ['survived'])
ydata = df.loc[:, ['survived']]

xdata = xdata.replace(np.nan, 0)
xdata.loc[xdata.cabin==-1,'cabin'] = 0
xdata.loc[xdata.embarked==-1,'embarked'] = 3

In [4]:
rd_seed = 333
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(xdata, ydata, test_size=0.25, random_state=rd_seed) 
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=rd_seed) 


In [5]:
models = {}

Decision Tree Classifier

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ParameterGrid
import sklearn.metrics as metrics
param_grid = {
    'max_depth': range(1,51), 
    'criterion': ['entropy', 'gini']
}
param_comb = ParameterGrid(param_grid)

val_acc = []
train_acc = []
for params in param_comb:
    dt = DecisionTreeClassifier(**params)
    dt.fit(Xtrain, ytrain)
    val_acc.append(metrics.accuracy_score(yval, dt.predict(Xval)))

best_params = param_comb[np.argmax(val_acc)]

dt = DecisionTreeClassifier(**best_params)
dt.fit(Xtrain, ytrain)
print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, dt.predict(Xtrain))))
print('accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, dt.predict(Xval))))
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, dt.predict(Xtest))))

models[metrics.accuracy_score(yval, dt.predict(Xval))] = dt

accuracy score (train): 0.798932
accuracy score (validation): 0.835106
accuracy score (test): 0.828000


Decision Tree Regressor / Random Forest Regressor / AdaBoost Regressor

In [7]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeRegressor
param_grid = {
    'max_depth': range(1,50)
}
param_comb = ParameterGrid(param_grid)
val_acc = []
for params in param_comb:
    dtr = DecisionTreeRegressor(**params)
    dtr.fit(Xtrain, ytrain)
    val_acc.append(math.sqrt(metrics.mean_squared_error(yval, dtr.predict(Xval))))
best_params = param_comb[np.argmin(val_acc)]
dtr = DecisionTreeRegressor(**best_params)
dtr.fit(Xtrain, ytrain)
print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, dtr.predict(Xtrain).round(0))))
print('accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, dtr.predict(Xval).round(0))))
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, dtr.predict(Xtest).round(0))))

models[metrics.accuracy_score(yval, dtr.predict(Xval).round(0))] = dtr

accuracy score (train): 0.830961
accuracy score (validation): 0.771277
accuracy score (test): 0.772000


In [8]:
from sklearn.ensemble import RandomForestRegressor
param_grid = {
    'n_estimators': range(1,100,5),
    'max_depth': range(1,5)
}
param_comb = ParameterGrid(param_grid)
val_acc = []
for params in param_comb:
    rfr = RandomForestRegressor(**params)
    rfr.fit(Xtrain, ytrain)
    val_acc.append(math.sqrt(metrics.mean_squared_error(yval, rfr.predict(Xval))))
best_params = param_comb[np.argmin(val_acc)]
rfr = RandomForestRegressor(**best_params)
rfr.fit(Xtrain, ytrain)
print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, rfr.predict(Xtrain).round(0))))
print('accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, rfr.predict(Xval).round(0))))
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, rfr.predict(Xtest).round(0))))

models[metrics.accuracy_score(yval, rfr.predict(Xval).round(0))] = rfr

accuracy score (train): 0.852313
accuracy score (validation): 0.819149
accuracy score (test): 0.832000


In [9]:
from sklearn.ensemble import AdaBoostRegressor
param_grid = {
    'n_estimators': range(1,100,5),
    'learning_rate': [0.01, 0.05, 0.1, 0.3, 0.5, 1]
}
param_comb = ParameterGrid(param_grid)
val_acc = []
for params in param_comb:
    abr = AdaBoostRegressor(**params)
    abr.fit(Xtrain, ytrain)
    val_acc.append(math.sqrt(metrics.mean_squared_error(yval, abr.predict(Xval))))
best_params = param_comb[np.argmin(val_acc)]
abr = AdaBoostRegressor(**best_params)
abr.fit(Xtrain, ytrain)
print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, abr.predict(Xtrain).round(0))))
print('accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, abr.predict(Xval).round(0))))
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, abr.predict(Xtest).round(0))))

models[metrics.accuracy_score(yval, abr.predict(Xval).round(0))] = abr

accuracy score (train): 0.816726
accuracy score (validation): 0.803191
accuracy score (test): 0.820000


KMeans

In [10]:
from sklearn.cluster import KMeans
param_grid = {
    'n_clusters': range(1,10),
    'init': ['k-means++', 'random'],
    'random_state': range(1,5)
}
param_comb = ParameterGrid(param_grid)
val_acc = []
for params in param_comb:
    knn = KMeans(**params).fit(Xtrain, ytrain)
    val_acc.append(metrics.accuracy_score(yval, knn.predict(Xval)))

best_params = param_comb[np.argmax(val_acc)]
knn = KMeans(**best_params).fit(Xtrain,ytrain)
print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, knn.predict(Xtrain))))
print('accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, knn.predict(Xval))))
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, knn.predict(Xtest))))

models[metrics.accuracy_score(yval, knn.predict(Xval))] = knn

accuracy score (train): 0.699288
accuracy score (validation): 0.670213
accuracy score (test): 0.624000


K Neighbors Regressor

In [11]:
from sklearn.neighbors import KNeighborsRegressor
param_grid = {
    'n_neighbors' : range(1,21),
    'p': range(1,5),
    'weights': ['uniform', 'distance']
}

param_comb = ParameterGrid(param_grid)

Xtrain_knn = Xtrain.copy() 
Xtest_knn = Xtest.copy() 
Xval_knn = Xval.copy()

Xtrain_knn = (Xtrain_knn - Xtrain_knn.min(axis=0))/(Xtrain_knn.max(axis=0) - Xtrain_knn.min(axis=0))
Xtest_knn = (Xtest_knn - Xtest_knn.min(axis=0))/(Xtest_knn.max(axis=0) - Xtest_knn.min(axis=0))
Xval_knn = (Xval_knn - Xval_knn.min(axis=0))/(Xval_knn.max(axis=0) - Xval_knn.min(axis=0))

val_acc = []
for params in param_comb:
    kNN = KNeighborsRegressor(**params).fit(Xtrain_knn, ytrain)
    val_acc.append(metrics.accuracy_score(yval, knn.predict(Xval_knn)))

best_params = param_comb[np.argmax(val_acc)]

kNN = KNeighborsRegressor(**best_params)
Xtest_knn.fillna(0, inplace=True)

kNN.fit(Xtrain_knn, ytrain)

print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, kNN.predict(Xtrain_knn).round(0))))
print('accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, kNN.predict(Xval_knn).round(0))))
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, kNN.predict(Xtest_knn).round(0))))

models[metrics.accuracy_score(yval, kNN.predict(Xval_knn).round(0))] = kNN

accuracy score (train): 0.973310
accuracy score (validation): 0.771277
accuracy score (test): 0.780000


Naive Bayes

In [12]:
from scipy.misc import logsumexp
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.preprocessing import OneHotEncoder

pclass = np.array(xdata['pclass']).reshape(-1,1)
x0 = OneHotEncoder(sparse = False).fit_transform(pclass)
age = np.array(xdata['age']).reshape(-1,1)
x1 = OneHotEncoder(sparse = False).fit_transform(age)
sibsp = np.array(xdata['sibsp']).reshape(-1,1)
x2 = OneHotEncoder(sparse = False).fit_transform(sibsp)
parch = np.array(xdata['parch']).reshape(-1,1)
x3 = OneHotEncoder(sparse = False).fit_transform(parch)
fare = np.array(xdata['fare']).reshape(-1,1)
x4 = OneHotEncoder(sparse = False).fit_transform(fare)
cabin = np.array(xdata['cabin']).reshape(-1,1)
x5 = OneHotEncoder(sparse = False).fit_transform(cabin)
embarked = np.array(xdata['embarked']).reshape(-1,1)
x6 = OneHotEncoder(sparse = False).fit_transform(embarked)
x7 = np.array(xdata.sex).reshape(-1,1)

data = np.concatenate([x0,x7,x1,x2,x3,x4,x5,x6], axis=1)
clf = BernoulliNB(fit_prior = False).fit(data, ydata)
pred = np.array([clf.predict(data)])
print('accuracy score: {0:.6f}'.format(metrics.accuracy_score(ydata, clf.predict(data))))

models[metrics.accuracy_score(ydata, clf.predict(data))] = clf

accuracy score: 0.794000


Linear Regression

In [13]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(Xtrain,ytrain)
print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, lr.predict(Xtrain).round(0))))
print('accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, lr.predict(Xval).round(0))))
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, lr.predict(Xtest).round(0))))

models[metrics.accuracy_score(yval, lr.predict(Xval).round(0))] = lr

accuracy score (train): 0.782918
accuracy score (validation): 0.813830
accuracy score (test): 0.812000


Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression
param_grid = {
    'penalty': ['l1', 'l2'],
    'random_state': range(1,5),
    'max_iter': range(1,100)
}
param_comb = ParameterGrid(param_grid)
val_acc = []
for params in param_comb:
    lgr = LogisticRegression(**params).fit(Xtrain, ytrain)
    val_acc.append(metrics.accuracy_score(yval, lgr.predict(Xval)))

best_params = param_comb[np.argmax(val_acc)]
lgr = LogisticRegression(**best_params).fit(Xtrain,ytrain)
print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, lgr.predict(Xtrain))))
print('accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, lgr.predict(Xval))))
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, lgr.predict(Xtest))))

models[metrics.accuracy_score(yval, lgr.predict(Xval))] = lgr


accuracy score (train): 0.788256
accuracy score (validation): 0.829787
accuracy score (test): 0.820000


In [15]:
evaluation = pd.read_csv('evaluation.csv')
ev = evaluation.drop(columns = ['ID', 'name', 'ticket', 'home.dest'])
string_cols = ev.select_dtypes(['object']).columns
for col in string_cols:
    ev[col] = ev[col].astype('category')
ev[string_cols] = ev[string_cols].apply(lambda x: x.cat.codes)
ev = ev.replace(np.nan, 0)
ev.loc[ev.cabin==-1,'cabin'] = 0
ev.loc[ev.embarked==-1,'embarked'] = 3
model = models[max(models)]
print("Nejlepsi model je ", model)
print("s presnosti ", (max(models)*100).round(0), "%")

if model == clf:
    pclass = np.array(ev['pclass']).reshape(-1,1)
    x0 = OneHotEncoder(sparse = False).fit_transform(pclass)
    age = np.array(ev['age']).reshape(-1,1)
    x1 = OneHotEncoder(sparse = False).fit_transform(age)
    sibsp = np.array(ev['sibsp']).reshape(-1,1)
    x2 = OneHotEncoder(sparse = False).fit_transform(sibsp)
    parch = np.array(ev['parch']).reshape(-1,1)
    x3 = OneHotEncoder(sparse = False).fit_transform(parch)
    fare = np.array(ev['fare']).reshape(-1,1)
    x4 = OneHotEncoder(sparse = False).fit_transform(fare)
    cabin = np.array(ev['cabin']).reshape(-1,1)
    x5 = OneHotEncoder(sparse = False).fit_transform(cabin)
    embarked = np.array(ev['embarked']).reshape(-1,1)
    x6 = OneHotEncoder(sparse = False).fit_transform(embarked)
    x7 = np.array(ev.sex).reshape(-1,1)

    data = np.concatenate([x0,x7,x1,x2,x3,x4,x5,x6], axis=1)
    survived = np.array(model.predict(data)).T
else:
    survived = model.predict(ev).round(0)
    
ID = evaluation.loc[:, ['ID']]
result = pd.concat([ID, pd.DataFrame(survived)], axis=1)
result.columns = ['ID', 'survived']
result.to_csv('result.csv')
display(result.head())


Nejlepsi model je  DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
s presnosti  84.0 %


,ID,survived
0,1000,0
1,1001,0
2,1002,0
3,1003,1
4,1004,0
